In [10]:
import plotly
import plotly.graph_objs as go
import pandas as pd
import numpy as np
# !pip install pandas

In [2]:
# Assume;
# OTR (mmoles O2 / L hr) = kLa *∆p = 0.19*kLa; ∆p is the partial pressure difference between
# the gas vs liquid phase.
# kLa (mmoles O2 / hour atm) = 0.95* (0.032 *A)^0.5 * N^0.5
# A is the aeration with a unit of liter per min
# N is the agitation with a unit of revolution per min

# The OUR at time t will increase exponentially until OUR is greater or equal to the maximum
# OTR that can be supported by the bioreactor. OUR(t) = OUR(t0) e^(0.2(t - t0)), in which the
# operator will change the fermentation feed scheme to stop the OUR increase.

# The initial OUR at T0 = 0.01 mmoles O2 / L hr. (1) The controller will first increase agitation
# from 30-150, once the agitation reaches the 150 maximum, the controller will then increase
# the aeration from 5000 to 10,000 L/min, until it reaches the 10,000 L/min maximum.

In [14]:
#start conditions
OUR_t0 = 0.01
A = 5000
N = 30
t = 0

#equations
kLa = 0.95*(0.032*A)**0.5*N**0.5
OTR = 0.19*kLa

#record the t,our,otr,N, and A at eatch time step
record = []

#step time by 1
for i in np.arange(0.0, 100.0,.01):
    t = i
    OUR = OUR_t0 * 2.71828**(0.2*(t-0))
    if OUR >= OTR:
        #increase agitation and check if it is at max or if OTR is greater than OUR
        if N < 150 and   OUR >= OTR:
            while N < 150 and OUR >= OTR:
                N = N + 1
                kLa = 0.95*(0.032*A)**0.5*N**0.5
                OTR = 0.19*kLa
        if N>=150 and A < 10000 and OUR >= OTR:
            while A < 10000 and OUR >= OTR:
                A = A + 1
                kLa = 0.95*(0.032*A)**0.5*N**0.5
                OTR = 0.19*kLa
        if OUR >= OTR:
            break
        
    record.append([t,OUR, OTR, N, A])
df = pd.DataFrame(record, columns=['t','OUR', 'OTR', 'N', 'A'])
df




,t,OUR,OTR,N,A
0,0.00,0.010000,12.505407,30,5000
1,0.01,0.010020,12.505407,30,5000
2,0.02,0.010040,12.505407,30,5000
3,0.03,0.010060,12.505407,30,5000
4,0.04,0.010080,12.505407,30,5000
...,...,...,...,...,...
4137,41.37,39.205782,39.205996,150,9829
4138,41.38,39.284272,39.285691,150,9869
4139,41.39,39.362920,39.363239,150,9908
4140,41.40,39.441724,39.442616,150,9948


In [13]:
from plotly.subplots import make_subplots

# Create subplots: 2 rows, 1 column
fig = make_subplots(rows=2, cols=1)

# Add traces
fig.add_trace(go.Scatter(x=df['t'], y=df['OUR'], mode='lines', name='OUR'), row=1, col=1)
fig.add_trace(go.Scatter(x=df['t'], y=df['OTR'], mode='lines', name='OTR'), row=1, col=1)

fig.add_trace(go.Scatter(x=df['t'], y=df['N'], mode='lines', name='N'), row=2, col=1)
fig.add_trace(go.Scatter(x=df['t'], y=df['A'], mode='lines', name='A', yaxis='y2'), row=2, col=1)

# Update xaxis properties
fig.update_xaxes(title_text='Time (hours)', row=1, col=1)
fig.update_xaxes(title_text='Time (hours)', row=2, col=1)

# Update yaxis properties
fig.update_yaxes(title_text='oxyen transfer/uptake rate (mmoles O2 / L hr)', row=1, col=1)
fig.update_yaxes(title_text='Agitation (rev/min)', row=2, col=1)
fig.update_yaxes(title_text='Aeration (L/min)', secondary_y=True, row=2, col=1)

# Update title and height
fig.update_layout(title_text='Subplots: OUR and OTR vs Time (Top), N and A vs Time (Bottom)', height=700)

fig.show()

Can adjust N or A

In [ ]:
#start conditions
OUR_t0 = 0.01
A = 5000
N = 30
t = 0

#equations
kLa = 0.95*(0.032*A)**0.5*N**0.5
OTR = 0.19*kLa

#record the t,our,otr,N, and A at eatch time step
record = []

#step time by 1
for i in np.arange(0.0, 100.0,.01):
    t = i
    OUR = OUR_t0 * 2.71828**(0.2*(t-0))
    if OUR >= OTR:
        #increase agitation and check if it is at max or if OTR is greater than OUR
        while OUR >= OTR:
            if OUR >= OTR and A>=10000 and N>=150:
                break
            if N < 150: 
                stir_diff = 0
                kLa_add_stir = 0.95*(0.032*A)**0.5*(N+1)**0.5
                OTR_add_stir = 0.19*kLa_add_stir
                stir_diff = OUR - OTR_add_stir
            if A < 10000:
                air_diff = 0
                kLa_add_air = 0.95*(0.032*A+1)**0.5*N**0.5
                OTR_add_air = 0.19*kLa_add_air
                air_diff = OUR - OTR_add_air
            #see  stir or air will get over OTR
            
            if stirr_diff > air_diff:
                N = N + 1
            else:
                A = A + 1

                
        if OUR >= OTR and A>=10000 and N>=150:
            break
        
    record.append([t,OUR, OTR, N, A])
